# Group Sales Campaign Data Files
My client has several files containing details of email campaigns sent to their customer base. The client wants to use the file to see if sales people (Owners) are increasing the number of customers they send campaigns to. Merely sending emails in successive campaigns to the same customers does not count as increasing the number of customers. The client wants to see that sales people are getting new customers and including them in campaigns.

### Input File Structure

* Each file contains records of one campaign
* The file name records the Campaign, Campaign Type, and Date
* Some campaigns do not have a campaign type, some do not have a date
* PLEASE NOTE: Actual client data has been replaced with fake data for upload to github

### Output Requested

The client would like a single csv file that can be opened in Excel and sorted with column headers (an Excel feature). This file is to include:

* the data extracted from the file names as data columns (see "Input File Structure," above)
* columns from within the files (Account Owner, Company, Contact [Name, Email, Phone #]), etc
* an additional Owner Cross-Check column that checks the owner against an Account Assignments file
* duplicate rows deleted so that successive campaign emails to the same customer are not counted
    - duplicate rows defined as having same owner, contact email, & campaign (not campaign type or date)

### Aggregate Data
In addition the client would like to see a list of how many customers each salesperson/owner has included in campaigns. This will be written to a separate .csv file.

In [1]:
import csv
import pandas as pd
import numpy as np
import glob
from datetime import datetime

### Put info from file names into columns
One file in the 'data' directory contains a list of Account Assignments. The rest of the files are records of campaigns that were sent out. 

Each of those file names contains a campaign code, date (or 'blank'), and for one campaign code, a sub-code.

In [2]:
allFiles = glob.glob("data\*.csv")
frame = pd.DataFrame()
frame_list = []
for fl in allFiles:
    file_details = fl[5:].split("-") # split out campaign, vertical/retail, & date from file name
    
    # skip over the Account_Assignments.csv file
    if file_details[0] == "Account":
        continue
        
    df = pd.read_csv(fl, index_col=0, header=0)
    df['Campaign'] = file_details[0]
    if file_details[0] == "cmdm":
        df['Campaign Type'] = file_details[1]  # This campaign has sub-types
        try:                                   # and may or may not have a date
            camp_date = datetime.strptime(file_details[2].split(".")[0], "%m%d%Y")
        except:
            camp_date = np.nan
        df['Campaign Date'] = camp_date
    else:
        df['Campaign Type'] = " "  # Other campaigns do not have sub-types...
        df['Campaign Date'] = file_details[1]   # so element [1] is the date.
    
    frame_list.append(df)
frame = pd.concat(frame_list)
frame.head()

,Email,Lead Status,First Name,Last Name,Phone Number,Owner ID,Last Activity Date,Last Contacted,Create Date,Associated Company Name,Create Date.1,First Contact Create Date,Campaign,Campaign Type,Campaign Date
Contact ID,,,,,,,,,,,,,,,
1291537,frannie@mapsy.com,NaN,Frannie,Smith,123-456-7890,Mark Smith,NaN,NaN,1/19/2016 18:40,Mapsy Inc,1/19/2016 19:03,1/19/2016 18:36,cmdm,allverticals,2018-06-04
1297424,johnhopkins@maxicom.com,NaN,John,Hopkins,123-456-7890,Claudia Franken,9/18/2018 18:46,9/18/2018 18:43,1/19/2016 18:43,Maxicom Global,1/19/2016 19:32,1/19/2016 18:27,cmdm,allverticals,2018-06-04
1297172,smooc.hanson@harlequin.com,NaN,Smooc,Hanson,123-456-7890,NaN,8/21/2018 17:35,NaN,1/19/2016 18:43,Harliquin Inc,2/24/2016 18:21,1/19/2016 18:34,cmdm,allverticals,2018-06-04
1265684,don.laguardia@peripheral.com,NaN,Don,Laguardia,123-456-7890,Fred Crew,9/14/2018 16:15,9/14/2018 16:13,1/19/2016 18:29,Peripheral Subjects,2/9/2016 19:52,1/19/2016 18:28,cmdm,allverticals,2018-06-04
1289499,matt.grossman@mapsy.com,NaN,Marla,Grossberg,123-456-7890,Sheri Stewart,NaN,NaN,1/19/2016 18:39,Mapsy Inc,1/19/2016 19:03,1/19/2016 18:36,cmdm,allverticals,2018-06-04


### Insert a column that cross-checks Owner ID with Owner identified in Account-Assignments file

1. Read the Account_Assignmens file into a separate dataframe
2. Rename columns to prepare for merge


In [3]:
df_asgn = pd.read_csv("data\Account-Assignments.csv", index_col=None, header=0)
df_asgn.columns = ['Associated Company Name','Owner']
df_asgn.head()

,Associated Company Name,Owner
0,Mapsy Inc.,Mark Smith
1,Maxicom Global,Claudia Franken
2,Harlequin Inc.,Joanne Jelico
3,Peripheral Subjects,Fred Crew


### Merge in assigned owners, and then arrange column order for easier viewing in delivered file.

In [4]:
full_df = pd.merge(frame, df_asgn, on = "Associated Company Name")
full_df= full_df.rename(columns = {"Owner": "Assigned Owner"})

# Put the Assigned Owner next to the Owner ID column
colnames = full_df.columns.tolist()
col_order = [0,1,2,3,4,5,15,6,7,8,9,10,11,12,13,14]
colnames = [ colnames[i] for i in col_order ]
full_df = full_df[colnames]
full_df.head()

,Email,Lead Status,First Name,Last Name,Phone Number,Owner ID,Assigned Owner,Last Activity Date,Last Contacted,Create Date,Associated Company Name,Create Date.1,First Contact Create Date,Campaign,Campaign Type,Campaign Date
0,johnhopkins@maxicom.com,NaN,John,Hopkins,123-456-7890,Claudia Franken,Claudia Franken,9/18/2018 18:46,9/18/2018 18:43,1/19/2016 18:43,Maxicom Global,1/19/2016 19:32,1/19/2016 18:27,cmdm,allverticals,2018-06-04
1,don.laguardia@peripheral.com,NaN,Don,Laguardia,123-456-7890,Fred Crew,Fred Crew,9/14/2018 16:15,9/14/2018 16:13,1/19/2016 18:29,Peripheral Subjects,2/9/2016 19:52,1/19/2016 18:28,cmdm,allverticals,2018-06-04


### Remove duplicate campaign emails, Write files

* Each salesperson (Owner) will have one record for email sent to any given email address for each campaign.
* The de-duplicated file will be written out to a .csv file as requested.
* Aggregation will be done for each salesperson/campaign to show how many contacts were made for that campaign. This is written into a separate summary .csv file.

In [7]:
full_df = full_df.drop_duplicates(subset=['Owner ID', 'Campaign', 'Email'], keep='first')
# write pandas dataframe to csv
full_df.to_csv("output\campaign-contacts.csv", sep=',')

grouped_df = full_df.groupby(['Owner ID', 'Campaign'])['Email'].count()
grouped_df.to_csv("output\campaign_summary.csv", sep=',')
grouped_df.head()

Owner ID         Campaign
Claudia Franken  cmdm        1
Fred Crew        cmdm        1
Name: Email, dtype: int64